<a href="https://colab.research.google.com/github/SLCFLAB/DL-Forecasting/blob/main/DL/day1/1_2_One_Step_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python

#### Requirements

In [1]:
!pip install darts
!pip install omegaconf
!pip install dropbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python.git

Cloning into 'Modern-Time-Series-Forecasting-with-Python'...
remote: Enumerating objects: 623, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 623 (delta 0), reused 0 (delta 0), pack-reused 616
Receiving objects: 100% (623/623), 196.35 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (369/369), done.
Updating files: 100% (91/91), done.


In [3]:
import os
path = os.getcwd() + '/Modern-Time-Series-Forecasting-with-Python/'
os.chdir(path)

In [4]:
import shutil
import joblib

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

from pathlib import Path

from src.forecasting.ml_forecasting import (
    MissingValueConfig,
    calculate_metrics,
)
from src.utils import plotting_utils
from tqdm.autonotebook import tqdm
from src.forecasting.ml_forecasting import calculate_metrics
from src.utils import ts_utils
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

/content/Modern-Time-Series-Forecasting-with-Python/src/utils/data_utils.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
os.makedirs("imgs/chapter_12", exist_ok=True)
os.makedirs("data/london_smart_meters/preprocessed", exist_ok=True)
os.makedirs("data/london_smart_meters/output", exist_ok=True)

preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

In [6]:
### upload 3 files from github

## Utility Functions

In [7]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        )
    )
    return fig

In [8]:
from itertools import cycle


def plot_forecast(pred_df, forecast_columns, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [
        "rgba(" + ",".join([str(c) for c in plotting_utils.hex_to_rgb(c)]) + ",<alpha>)"
        for c in px.colors.qualitative.Plotly
    ]
    act_color = colors[0]
    colors = cycle(colors[1:])
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].energy_consumption,
            mode="lines",
            line=dict(color=act_color.replace("<alpha>", "0.9")),
            name="Actual Consumption",
        )
    )
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines",
                line=dict(dash="dot", color=next(colors).replace("<alpha>", "1")),
                name=display_col,
            )
        )
    return fig

def highlight_abs_min(s, props=''):
    return np.where(s == np.nanmin(np.abs(s.values)), props, '')

## Reading the data

In [9]:
import dropbox
from io import BytesIO

In [18]:
token = "sl.BhF3htSUGKa56RO6jbuz7Fzt_OQotDJM1UOD_T6c7yTbXWfLGZFQv0DFte1eq9uoKd6AgqwrLg2I140XrHpCHY2JhZK0eyG0BlkIaVvaHM5btxmV3_gQEDQJaLi8UjxA2GIwA7M"
dbx = dropbox.Dropbox(token)
path1 = "/selected_blocks_train_missing_imputed_feature_engg.parquet"
path2 = "/selected_blocks_val_missing_imputed_feature_engg.parquet"

_, res = dbx.files_download(path1)
assert res.status_code == 200, 'Could not download file'
train_df = pd.read_parquet(BytesIO(res.content))

_, res = dbx.files_download(path2)
assert res.status_code == 200, 'Could not download file'
test_df = pd.read_parquet(BytesIO(res.content))

In [ ]:
# try:
#     #Reading the missing value imputed and train test split data
#     train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
#     # Read in the Validation dataset as test_df so that we predict on it
#     test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")

# except FileNotFoundError:
#     display(HTML("""
#     <div class="alert alert-block alert-warning">
#     <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
#     </div>
#     """))

In [19]:
target = "energy_consumption"
index_cols = ["LCLid", "timestamp"]

In [20]:
# Setting the indices
train_df.set_index(index_cols, inplace=True, drop=False)
test_df.set_index(index_cols, inplace=True, drop=False)

### Loading the Single Step ML Forecast
- upload 3 pickle files from github

In [21]:
try:
    single_step_ahead_ml_fc_df = pd.read_pickle(output/"ml_single_step_prediction_val_df.pkl")
    single_step_ahead_ml_metrics_df = pd.read_pickle(output/"ml_single_step_metrics_val_df.pkl")
    single_step_ahead_ml_agg_metrics_df = pd.read_pickle(output/"ml_single_step_aggregate_metrics_val.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Forecasting with ML in Chapter08
    </div>
    """))

# Running the RNN on a Sample Household

## Selecting the sample data and metrics

In [22]:
sample_train_df = train_df.xs("MAC000193")
sample_test_df = test_df.xs("MAC000193")
# Creating a pred_df with actuals
pred_df = pd.concat([sample_train_df[[target]], sample_test_df[[target]]])

Split Train into Train and Validation and combine everything together into a single dataframe

In [23]:
sample_val_df = sample_train_df.loc["2013-12"]
sample_train_df = sample_train_df.loc[:"2013-11"]

sample_train_df['type'] = "train"
sample_val_df['type'] = "val"
sample_test_df['type'] = "test"
sample_df = pd.concat([sample_train_df[[target, "type"]], sample_val_df[[target, "type"]], sample_test_df[[target, "type"]]])
sample_df.head()

,energy_consumption,type
timestamp,,
2012-01-01 00:00:00,0.368,train
2012-01-01 00:30:00,0.386,train
2012-01-01 01:00:00,0.170,train
2012-01-01 01:30:00,0.021,train
2012-01-01 02:00:00,0.038,train


In [24]:
metric_record = []
metric_record += (
    single_step_ahead_ml_metrics_df.loc[single_step_ahead_ml_metrics_df.LCLid == "MAC000193"]
    .drop(columns="LCLid")
    .to_dict(orient="records")
)
metric_record

[{'Algorithm': 'Lasso Regression',
  'MAE': 0.15978845101080352,
  'MSE': 0.07430985710387804,
  'MASE': 1.245259122197505,
  'Forecast Bias': 3.7566336422042634},
 {'Algorithm': 'XGB Random Forest',
  'MAE': 0.1646268218755722,
  'MSE': 0.08218114823102951,
  'MASE': 1.2829653024673462,
  'Forecast Bias': 9.215143322944641},
 {'Algorithm': 'LightGBM',
  'MAE': 0.14886508634822698,
  'MSE': 0.06913165778511937,
  'MASE': 1.1601314461663779,
  'Forecast Bias': 3.862088397030192}]

## Loading the necessary classes

In [25]:
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.models import SingleStepRNNConfig, SingleStepRNNModel
import pytorch_lightning as pl
import torch
# For reproduceability set a random seed
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

### Creating the datamodule which splits and formats the data into windows

In [26]:
datamodule = TimeSeriesDataModule(data = sample_df[[target]],
        n_val = sample_val_df.shape[0],
        n_test = sample_test_df.shape[0],
        window = 48, # giving enough memory to capture daily seasonality
        horizon = 1, # single step
        normalize = "global", # normalizing the data
        batch_size = 32,
        num_workers = 0)
datamodule.setup()

### Setting the config for the RNN and initializing the model

In [27]:
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=1,
    hidden_size=128,
    num_layers=3,
    bidirectional=True,
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)

### Manual Inspection

In [28]:
# Getting a batch from the train_dataloader
for batch in datamodule.train_dataloader():
    x, y = batch
    break
print("Shape of x: ",x.shape)
print("Shape of y: ",y.shape)

Shape of x:  torch.Size([32, 48, 1])
Shape of y:  torch.Size([32, 1, 1])


In [29]:
# Running the batch through the model
# We expect two outputs - the first one is the forecast and second is the corresponding target
y_hat, y = model(batch)
print("Shape of y_hat: ",y_hat.shape)
print("Shape of y: ",y.shape)

Shape of y_hat:  torch.Size([32, 48, 1])
Shape of y:  torch.Size([32, 48, 1])


In [30]:
# Calculating the loss
l = model.loss(y_hat, y)
print(l)

tensor(0.9786, grad_fn=<MseLossBackward0>)


### Full Training

In [31]:
trainer = pl.Trainer(
    #auto_select_gpus=True,
    #gpus=-1,
    min_epochs=5,
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor="valid_loss", patience=3)],
)
trainer.fit(model, datamodule)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 231 K 
1 | fc   | Linear  | 257   
2 | loss | MSELoss | 0     
---------------------------------
231 K     Trainable params
0         Non-trainable params
231 K     Total params
0.926     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [32]:
# Removing artifacts created during training
shutil.rmtree("lightning_logs")

### Prediction

In [33]:
pred = trainer.predict(model, datamodule.test_dataloader())
# pred is a list of outputs, one for each batch
pred = torch.cat(pred).squeeze().detach().numpy()
# Apply reverse transformation because we applied global normalization
pred = pred * datamodule.train.std + datamodule.train.mean
pred_df_ = pd.DataFrame({rnn_config.rnn_type: pred}, index=sample_test_df.index)
pred_df = pred_df.join(pred_df_)
metrics = calculate_metrics(sample_test_df[target], pred_df_[rnn_config.rnn_type], rnn_config.rnn_type, pd.concat([sample_train_df[target],sample_val_df[target]]))
metric_record.append(metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [34]:
formatted = pd.DataFrame(metric_record).style.format({"MAE": "{:.4f}",
                          "MSE": "{:.4f}",
                          "MASE": "{:.4f}",
                          "Forecast Bias": "{:.2f}%"})
formatted.highlight_min(color='lightgreen', subset=["MAE","MSE","MASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,MASE,Forecast Bias
0,Lasso Regression,0.1598,0.0743,1.2453,3.76%
1,XGB Random Forest,0.1646,0.0822,1.2830,9.22%
2,LightGBM,0.1489,0.0691,1.1601,3.86%
3,RNN,0.1622,0.0807,1.2638,8.41%


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 2.3 MB/s eta 0:00:00


In [38]:
fig = plot_forecast(pred_df, forecast_columns=[rnn_config.rnn_type], forecast_display_names=[rnn_config.rnn_type])
fig = format_plot(fig, title=f"{rnn_config.rnn_type}: MAE: {metrics['MAE']:.4f} | MSE: {metrics['MSE']:.4f} | MASE: {metrics['MASE']:.4f} | Bias: {metrics['Forecast Bias']:.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_12/rnn.png")
fig.show()

# Running LSTMs and GRUs on a Sample Household

## LSTM

In [41]:
rnn_config = SingleStepRNNConfig(
    rnn_type="LSTM",
    input_size=1,
    hidden_size=128,
    num_layers=3,
    bidirectional=True,
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)

trainer = pl.Trainer(
    #auto_select_gpus=True,
    #gpus=-1,
    #auto_lr_find=True,
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor="valid_loss", patience=3)],
)
trainer.fit(model, datamodule)
# Removing artifacts created during training
shutil.rmtree("lightning_logs")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type    | Params
---------------------------------
0 | rnn  | LSTM    | 924 K 
1 | fc   | Linear  | 257   
2 | loss | MSELoss | 0     
---------------------------------
924 K     Trainable params
0         Non-trainable params
924 K     Total params
3.700     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [42]:
pred = trainer.predict(model, datamodule.test_dataloader())
# pred is a list of outputs, one for each batch
pred = torch.cat(pred).squeeze().detach().numpy()
# Apply reverse transformation because we applied global normalization
pred = pred * datamodule.train.std + datamodule.train.mean
pred_df_ = pd.DataFrame({rnn_config.rnn_type: pred}, index=sample_test_df.index)
pred_df = pred_df.join(pred_df_)
metrics = calculate_metrics(sample_test_df[target], pred_df_[rnn_config.rnn_type], rnn_config.rnn_type, pd.concat([sample_train_df[target],sample_val_df[target]]))
metric_record.append(metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [43]:
formatted = pd.DataFrame(metric_record).style.format({"MAE": "{:.4f}",
                          "MSE": "{:.4f}",
                          "MASE": "{:.4f}",
                          "Forecast Bias": "{:.2f}%"})
formatted.highlight_min(color='lightgreen', subset=["MAE","MSE","MASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,MASE,Forecast Bias
0,Lasso Regression,0.1598,0.0743,1.2453,3.76%
1,XGB Random Forest,0.1646,0.0822,1.2830,9.22%
2,LightGBM,0.1489,0.0691,1.1601,3.86%
3,RNN,0.1622,0.0807,1.2638,8.41%
4,LSTM,0.1581,0.0826,1.2324,14.28%


In [45]:
fig = plot_forecast(pred_df, forecast_columns=[rnn_config.rnn_type], forecast_display_names=[rnn_config.rnn_type])
fig = format_plot(fig, title=f"{rnn_config.rnn_type}: MAE: {metrics['MAE']:.4f} | MSE: {metrics['MSE']:.4f} | MASE: {metrics['MASE']:.4f} | Bias: {metrics['Forecast Bias']:.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_12/lstm.png")
fig.show()

## GRU

In [46]:
rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=1,
    hidden_size=128,
    num_layers=3,
    bidirectional=True,
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)

trainer = pl.Trainer(
    #auto_select_gpus=True,
    #gpus=-1,
    #auto_lr_find=True,
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor="valid_loss", patience=3)],
)
trainer.fit(model, datamodule)
# Removing artifacts created during training
shutil.rmtree("lightning_logs")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type    | Params
---------------------------------
0 | rnn  | GRU     | 693 K 
1 | fc   | Linear  | 257   
2 | loss | MSELoss | 0     
---------------------------------
693 K     Trainable params
0         Non-trainable params
693 K     Total params
2.775     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [47]:
pred = trainer.predict(model, datamodule.test_dataloader())
# pred is a list of outputs, one for each batch
pred = torch.cat(pred).squeeze().detach().numpy()
# Apply reverse transformation because we applied global normalization
pred = pred * datamodule.train.std + datamodule.train.mean
pred_df_ = pd.DataFrame({rnn_config.rnn_type: pred}, index=sample_test_df.index)
pred_df = pred_df.join(pred_df_)
metrics = calculate_metrics(sample_test_df[target], pred_df_[rnn_config.rnn_type], rnn_config.rnn_type, pd.concat([sample_train_df[target],sample_val_df[target]]))
metric_record.append(metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [48]:
formatted = pd.DataFrame(metric_record).style.format({"MAE": "{:.4f}",
                          "MSE": "{:.4f}",
                          "MASE": "{:.4f}",
                          "Forecast Bias": "{:.2f}%"})
formatted.highlight_min(color='lightgreen', subset=["MAE","MSE","MASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,MASE,Forecast Bias
0,Lasso Regression,0.1598,0.0743,1.2453,3.76%
1,XGB Random Forest,0.1646,0.0822,1.2830,9.22%
2,LightGBM,0.1489,0.0691,1.1601,3.86%
3,RNN,0.1622,0.0807,1.2638,8.41%
4,LSTM,0.1581,0.0826,1.2324,14.28%
5,GRU,0.1603,0.0720,1.2493,3.61%


In [50]:
fig = plot_forecast(pred_df, forecast_columns=[rnn_config.rnn_type], forecast_display_names=[rnn_config.rnn_type])
fig = format_plot(fig, title=f"{rnn_config.rnn_type}: MAE: {metrics['MAE']:.4f} | MSE: {metrics['MSE']:.4f} | MASE: {metrics['MASE']:.4f} | Bias: {metrics['Forecast Bias']:.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_12/gru.png")
fig.show()

In [52]:
fig = plot_forecast(pred_df, forecast_columns=["LSTM", "GRU"], forecast_display_names=["LSTM", "GRU"])
fig = format_plot(fig, title=f"LSTM, and GRU")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.update_traces(line=dict(dash="dash"), selector= dict(name="GRU"))
#fig.write_image("imgs/chapter_12/lstm_gru.png")
fig.show()

In [ ]:
# pred_df.to_pickle(output/"dl_single_step_prediction_val_df_MAC000193.pkl")
# joblib.dump(metric_record, output/"dl_single_step_metrics_val_df_MAC000193.pkl")

['data\\london_smart_meters\\output\\dl_single_step_metrics_val_df_MAC000193.pkl']